In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from pgmpy.estimators import K2Score
from rpy2.robjects import conversion, default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import rpy2.robjects as ro
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import BayesianEstimator as BE
from pgmpy.estimators import BDeuScore
from pgmpy.estimators import BicScore
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score
pandas2ri.activate()
from sklearn.metrics import f1_score

In [6]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix

def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix.loc[index,col]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list  
def print_dag(dag,all_node=False):
    if not all_node:
        if dag.has_node("label"):
            for edges in dag.get_ancestral_graph("label").edges:
                print(edges[0],"->",edges[1],";")
        else:
            print("label have not ancestor")
            for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    else:
         for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    return 
            
def model_auc(prob,y_true):
    y_prob = []
    num = 0
    for i in range(len(prob)):
        y_prob.append(prob.iloc[i,y_true[i]])
    if y_predic[i]==y_true[i]:
        num+=1
    fpr, tpr, thresholds = roc_curve(y_true,y_predic)
    
    return auc(fpr, tpr)
def to_strength_list(edges_strength_martix):
    str_list = []
    for col in edges_strength_martix.columns:
        for idx in edges_strength_matrix.columns:
            #不等于才加
            if edges_strength_matrix[col][idx]!=0:
                str_list.append([idx,col,edges_strength_matrix[col][idx]])
    return str_list

#根据互信息的分数计算黑名单
def caculate_hos_bl(data,bl):    
    mi_bl = []
    h_id = 6
    for col in data.columns:
        for index in data.columns:
            mi_score = mutual_info_score(data.loc[:,index],data.loc[:,col])
            if col==index:
                continue
            ##互信息分数，如果两个特征的互信息分数小雨0.02的话就将它加入黑名单
            if mi_score<0.02:
                mi_bl.append([index,col])
    h_bl=pd.DataFrame(mi_bl,columns=["from","to"])
    h_bl=pd.concat([h_bl,bl])
   
    return h_bl
def cross_validation(ed,DB):
    
    #cid客户端号，寻找相应的训练数据
    auc_score = []
    recall = []
    acc = []
    f1 = []
    for i in range(len(DB)): 
        
        model_i= BayesianNetwork(ed.edges)
        #创建边
        data = DB[i]
        
        f_1 = model_i.nodes
        
        f_2 = data.columns
        #所有mode有的节点
        drop_name = []
        for j in f_2:
            if not j in f_1:
                drop_name.append(j)        
        data = data.drop(columns = drop_name,axis = 1)
        
        
        #创建离散变量说明
        this_dict = []

        for key,value in zip(state_names.keys(),state_names.values()):
            
            if key in f_1:

                this_dict.append((key,value))
        this_dict = dict(this_dict)
         #训练模型
        
        
        train_data,test_data = train_test_split(data,test_size=0.3,random_state=2)
        #得到y-true
        y_true = test_data.iloc[:,0]
        #测试集
        model_i.fit(train_data,estimator=BE,complete_samples_only=True,state_names=this_dict)
        
        #丢弃label获得
        
        test_data.drop(columns="label",axis =1,inplace=True)
      
        #将预测的值从df转换为list形式
        
        #在predict的时候不知道什么会影响结果
        y_pred = list(model_i.predict(test_data).iloc[:,0])
       
       #模型预测
        acc.append(accuracy_score(y_true,y_pred))
        recall.append(recall_score(y_true, y_pred))
        auc_score.append(roc_auc_score(y_true,y_pred))
        f1.append(f1_score(y_true,y_pred))
        
    return acc,recall,auc_score,f1
def dag_score(dag,DB):
    score =  []
    
    for data in DB:
        #判断在每个网路之间分数
        
#                 k2 = K2Score(data)
#         score.append(k2.score(dag))
        bic = BicScore(data)
        score.append(bic.score(dag))
    score = np.array(score)
    return score
def net_score(ed,data):
    

        
    model_i= BayesianNetwork(ed.edges)

        
    f_1 = model_i.nodes
        
    f_2 = data.columns
    #所有mode有的节点
    drop_name = []
    for j in f_2:
        if not j in f_1:
            drop_name.append(j)  
   
    data = data.drop(columns = drop_name,axis = 1)
        
    #创建离散变量说明
    this_dict = []

    for key,value in zip(state_names.keys(),state_names.values()):
            
        if key in f_1:

            this_dict.append((key,value))
    this_dict = dict(this_dict)
        #训练模型
        
    train_data,test_data = train_test_split(data,test_size=0.3,random_state=2)
    #得到y-true
    
    y_true = test_data.iloc[:,0]
    
    model_i.fit(train_data,estimator=BE,complete_samples_only=True,state_names=this_dict)
        
    #丢弃label获得,test_data
    test_data.drop(columns="label",axis =1,inplace=True)
      
    #将预测的值从df转换为list形式
        
    #在predict的时候不知道什么会影响结果
    y_pred = list(model_i.predict(test_data).iloc[:,0])
       
    #模型预测
    acc = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    
        
    return acc,recall,auc,f1

In [7]:
#R函数定义部分
ro.r("""
       r_sturct_study<-function(self_r_data,bl,wl){
            library(bnlearn)
            datacols = names(self_r_data)

        
            
            for (i in 1:ncol(self_r_data)){
                if(i>45)
                {
                    self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
                }
                else if(datacols[i]=="race")
                {
                    self_r_data[,"race"] <- factor(self_r_data[,"race"],levels = c(1,2,3,4,5))
                }
                else{
                    self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
                 }
            }
      
            dag = hc(self_r_data,score=scr,blacklist =bl,whitelist=wl)
            ##dag = tabu(self_r_data,score=scr,blacklist =bl,whitelist=wl)
          
            arcs = arc.strength(dag,self_r_data)
            return(arcs)
        }
""")

#R函数定义部分
ro.r("""
       r_sturct_study_fit<-function(self_r_data,bl,wl){
            library(bnlearn)
            datacols = names(self_r_data)

        
            
            for (i in 1:ncol(self_r_data)){
                if(i>45)
                {
                    self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]],levels = c(0,1))
                }
                else if(datacols[i]=="race")
                {
                    self_r_data[,"race"] <- factor(self_r_data[,"race"],levels = c(1,2,3,4,5))
                }
                else{
                    self_r_data[,datacols[i]] <- factor(self_r_data[,datacols[i]])
                 }
            }
               
            dag = hc(self_r_data,score=scr,blacklist=bl)
            ##dag = tabu(self_r_data,score=scr,blacklist =bl)

            arcs = arc.strength(dag,self_r_data)
            return(arcs)
        }
""")

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x0000024BAD822A80> [RTYPES.CLOSXP]
R classes: ('function',)

In [8]:
class client():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid):
        #black 用R语言的存储方式存储·
        self.data = data
        self.cid = cid
        #初始化创建一个空的图
        self.model = DAG()
        feature = data.columns
        self.edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    def bl_update(self,bl):
        #在实例化客户端之后，根据自己数据的互信息值创建黑名单
        self.black_list = caculate_hos_bl(self.data,bl)
    def get_model(self):
        #获得本地客户端网络结构
        return self.model
    def fit(self):
        #=
        data,t_data = train_test_split(self.data,test_size=0.3,random_state=2)
        self_r_data = ro.pandas2ri.py2rpy(data)
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
        
        
        arc_strength = ro.r['r_sturct_study_fit'](self_r_data,self_r_black_list)
        arc_strength=ro.pandas2ri.rpy2py_dataframe(arc_strength)
        self.arc_strength=  arc_strength
        #创建新的模型，并添加结果
        self.model = DAG()
        print("客户端",self.cid+1,"开始fit")
        for i in range(len(arc_strength["from"])):
            
            #汇集频数
            self.model.add_edge(arc_strength["from"][i],arc_strength["to"][i])
          
            
#            #汇集强度转换为矩阵
            self.edges_strength.loc[[arc_strength["from"][i]],[arc_strength["to"][i]]]=arc_strength["strength"][i]
        #计算网路得分
        bic = BicScore(self.data)
        self.score=bic.score(self.model)
        return 1
    def set_parment(self):
        #从服务器中心get到网络结构
        #初始化model
        model.edges
    
        return 
    def update_model(self,wl):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
        data,t_data = train_test_split(self.data,test_size=0.3,random_state=2)
        
        self.model = DAG()
        wl_r =ro.conversion.py2rpy(wl)
        #客户端将自身的黑名单转换为R数据模式传进函数里
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
        self_r_data = ro.pandas2ri.py2rpy(data)
        #调用R函数，并接收返回值
        
        arc_strength_r = ro.r['r_sturct_study'](self_r_data,self_r_black_list,wl_r)
        
        self.arc_strength=ro.pandas2ri.rpy2py_dataframe(arc_strength_r)
        print("客户端",self.cid,"开始update")

        for i in range(len(self.arc_strength["from"])):
            #转换自己的网络
            self.model.add_edge(self.arc_strength["from"][i],self.arc_strength["to"][i])
            #生成新的强度列表
            self.edges_strength.loc[[self.arc_strength["from"][i]],[self.arc_strength["to"][i]]]=self.arc_strength["strength"][i]
        
        bic = BicScore(self.data)
        self.score=bic.score(self.model)
        
class sever():
    def __init__(self):
        pass
        
    def Fd_caculate(self,edges_strength_matrix,round_num):
        #聚合方法
        #聚合的方式改为计算强度的阈值，并返回一个有无环图
        
        #变为list，并且设置为df格式
        arc_ls = pd.DataFrame(to_strength_list(edges_strength_matrix.astype(np.float16)),columns=["from","to","strength"])
        #from和to的df格式并按大小排列
        arc_ls = arc_ls.sort_values(by="strength")
        #1.按照10个选择，但是白名单的列表应该不断的成长，而不是固定的数值
        #在每次讲融合DAG加入网络之后，先检验是否有这条边，如果有则跳过
    
        i = 0
        index = 0

        fuseDAG = DAG()
        while i <round_num*6:
            arc = (arc_ls.loc[i+index,"from"],arc_ls.loc[i+index,"to"])
            ##防止图的有环结构
            if fuseDAG.has_edge(arc[1],arc[0]):
                i=i+1
                continue
            fuseDAG.add_edge(arc[0],arc[1])
            
         
            i+=1
        #检查边是否有环，并且破环
        print("检查环")
        for root in fuseDAG.nodes():
            node_list=fuseDAG.get_ancestral_graph(root).nodes()
            for nd in node_list:
                if fuseDAG.has_edge(root,nd):
                    print(root,nd)
                    fuseDAG.remove_edge(root,nd)
        return  fuseDAG

In [9]:
DB_1 = pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
# DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/tenSiteDTDisc.csv").drop(columns=["Unnamed: 0","creatinine","patientunitstayid"],axis = 1)

DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)
DB_1.fillna(1)
# 数据分割
# hos_id = [420,142,122,435,390,227,195,243,403,141]
hos_id = [420,142,122,435,390,227,144,140,396,141]##性能最好
# hos_id = [420,142,122,435,390,382,144,140,396,141]
# hos_id = [420,142,122,384,390,227,144,140,396,141]
# hos_id = [420,142,122,435,390,227,144,384,143,403]
# hos_id = [420,142,122,435,384,143,403,195,382,243]##性能最后后六个是外部验证
client_data = []
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))
    
feature = client_data[0].columns

#设置黑名单
bl =[]
for i in feature:
    bl.append(["label",i])
    bl.append([i,"sex"])
    bl.append([i,"race"])
    bl.append([i,"bmi"])
    bl.append([i,"age"])
bl = pd.DataFrame(bl,columns=["from","to"])
# bl_r = ro.conversion.py2rpy(bl)
# globalenv['bl_r'] = bl_r
#计算样本量比重
D = 19388
D_W = []
for i in range(10):
    D_W.append(len(client_data[i])/D)
D_W = np.float16(D_W)

#将所有的离散状态记录下来，以字典的方式
state_names  ={
    "lable":[0,1],
    "age":[1,2,3,4,5,6,7,8],
    "sex":[1,2],
    "race":[1,2,3,4,5],
    "bmi":[1,2,3,4],
    "temperature":[1,2,3],
    "heartrate":[1,2,3],
    "respiration":[1,2,3],
    "SBP":[1,2,3,4,5],
    "DBP":[1,2,3,4,5],
    "paSystolic":[1,2,3],
    "paDiastolic":[1,2,3],
    "paMean":[1,2,3],
}
LAB_list = []
for i in range(1,61):
    LAB_list.append(("LAB"+str(i),[1,2,3]))
COM_list = []
for i in range(1,12):
    COM_list.append(("COM"+str(i),[0,1]))
PRO_list = []
for i in range(1,10):
    COM_list.append(("PRO"+str(i),[0,1]))
MED_list = []
for i in range(1,55):
    MED_list.append(("MED"+str(i),[0,1]))
state_names.update(LAB_list)
state_names.update(COM_list)
state_names.update(PRO_list)
state_names.update(MED_list)
BN_list = []
for i in range(1,11):
    BN_list.append("BN"+str(i))

In [10]:
all_score_init=[]
all_score_list=[]
fuse_dag_list = []
dag_struct_init_list = []
pandas2ri.activate()
scr_l=['fnml','bdla','mbde','bds','bic','loglik','bde', 'mbde', 'k2', 'qnml']
for scr in scr_l:
    if scr!="loglik":
        continue
    fuseDAG = DAG()
    
    globalenv['scr'] = scr

  
    # 结构训练
    train_round = 10
    round_num = 1
    client_num = 10
    clients = []

    for i in range(10):
    # 创建客户端，并且传入数据和结构学习模型
        clients.append(client(client_data[i],i))
    # 初始化服务器
    sever_1 = sever()
    #分别创建一个0/1矩阵和强度矩阵
    edges_strength_matrix_list =[]
    #记录边的频数
    global_edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    #记录边的强度
    edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    
    global_edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)

    score_list = []

    #先本地客户端建立初始模型,并且计算每个网络的初始性能
    print("计算每个网络的初始性能")
    for i in range(client_num):
        clients[i].bl_update(bl)
        clients[i].fit()
        dag_struct_init_list.append(clients[i].model)
        score_list.append(net_score(clients[i].model,data=clients[i].data))
    init_score = pd.DataFrame(score_list,index=BN_list,columns=["acc",'recall','auc','f1']).T
    all_score_init.append(init_score)
    #正式循环
    print("开始训练")
    while(round_num<train_round):
        #重置矩阵,设置评分
        global_edges_matrix.iloc[:,:] = 0
        edges_strength_matrix.iloc[:,:] = 0
        #get到每个客户端边强度的总和
        arc_strength_list = []
        arc_weight_list = []
        dag_score_list= []
        #开始聚集客户端边的信息

        for i in range(client_num):
            #将客户端最好的模型边信息转换为矩阵形式
            global_edges_matrix+=to_matrix(clients[i].model.edges(),feature)
            dag_score_list.append(clients[i].score)
            #将矩阵转换为列表形式，并strength按列合并数据
        #在这里计算边的强度
        #进行分数的权重
        dag_score_list=np.array(dag_score_list)
        min_score = min(dag_score_list)
#         #看下输出结构的分数
#         for i in range(len(dag_score_list)):
#             dag_score_list[i]=dag_score_list[i]-min_score+1
#             for j in range(len(clients[i].arc_strength)):
#                 print(clients[i].arc_strength.sort_values(by="strength",ascending=False).iloc[j,:])

    
        #汇集带权重的边的信息
        for i in range(client_num):
            #边的强度*数据量权重*网络分数/遍的个数 
    #         print(get_label_str(clients[i]))

            global_edges_strength_matrix+=clients[i].edges_strength

            edges_strength_matrix+=clients[i].edges_strength*D_W[i]*dag_score_list[i]/len(clients[i].model.edges())

        print("转换")
        #将矩阵转换为，一个from->to + str 的表


        global_edges_strength_matrix_list = to_strength_list(global_edges_strength_matrix)
        print("输出未加权只是相加的强度")


        edges_strength_matrix_list.append(edges_strength_matrix)
    #     print("")
    #     for i in range(len(edges_strength_matrix_list)):
    #         print(edges_strength_matrix_list.iloc[i,:])

        fuseDAG = sever_1.Fd_caculate(edges_strength_matrix,round_num)
        
        
        print("转换完毕")
        #输出一下混合的结果
        for j in fuseDAG.edges:
            print(j[0],"->",j[1],";")



        #得到fuseDAG之后，观察整个网络评分的变化

        print("第",round_num+1,"次迭代")
        #跟新客户端

        #设置白名单
        wl = []
        for edge in fuseDAG.edges():
            wl.append([edge[0],edge[1]])

        wl = pd.DataFrame(wl,columns=["from","to"])
        print("传递客户端信息")
        for i in range(client_num):

            clients[i]. update_model(wl)

        round_num+=1
    fuse_dag_list.append(fuseDAG)
    fuse_score_list = cross_validation(fuseDAG,client_data)
    fuse_score = pd.DataFrame(fuse_score_list,columns=BN_list,index=["acc",'recall','auc','f1'])
    all_score_list.append(fuse_score)

    
    
    



计算每个网络的初始性能
客户端 1 开始fit


  0%|          | 0/1284 [00:00<?, ?it/s]

客户端 2 开始fit


  0%|          | 0/489 [00:00<?, ?it/s]

客户端 3 开始fit


...

  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/428 [00:00<?, ?it/s]

In [1]:
all_score_init[0]

NameError: name 'all_score_init' is not defined

In [149]:
print_dag(dag_struct_init_list[2])

LAB42 -> COM8 ;
LAB39 -> LAB42 ;
LAB19 -> label ;
LAB11 -> LAB42 ;
LAB2 -> LAB39 ;
LAB2 -> LAB42 ;
COM11 -> label ;
LAB3 -> COM11 ;
LAB24 -> LAB39 ;
age -> LAB19 ;
COM7 -> COM3 ;
COM7 -> COM10 ;
COM3 -> COM10 ;
COM8 -> label ;
COM8 -> COM7 ;
COM8 -> COM10 ;
COM10 -> label ;
COM10 -> COM11 ;


In [150]:
print_dag(clients[2].model)

COM4 -> label ;
LAB42 -> COM8 ;
LAB7 -> LAB11 ;
LAB39 -> LAB42 ;
LAB19 -> label ;
LAB2 -> LAB39 ;
LAB2 -> LAB42 ;
LAB11 -> LAB42 ;
COM11 -> label ;
LAB24 -> LAB39 ;
COM1 -> label ;
age -> label ;
age -> COM1 ;
age -> LAB19 ;
COM7 -> label ;
COM7 -> COM3 ;
COM7 -> COM10 ;
COM3 -> label ;
COM3 -> COM10 ;
COM8 -> label ;
COM8 -> COM7 ;
COM8 -> COM10 ;
COM10 -> label ;
COM10 -> COM11 ;


In [13]:
def caculate_label_par(cl,cid):
    dag = DAG()
    for i in cl.get_parents("label"):
        dag.add_edge(i,"label")
    out_data=client_data[cid]
    for i in dag.edges():
        print(i[0],"->",i[1],";")
    return  net_score(dag,out_data),dag
def caculate_label_ancsetor(cl,cid):

    dag = cl.get_ancestral_graph("label")
       
    for i in dag.edges():
        print(i[0],"->",i[1],";")
    out_data=client_data[cid]

    return  net_score(dag,out_data),dag

In [19]:
c_id = 9

In [20]:
caculate_label_par(dag_struct_init_list[c_id ],c_id )

COM10 -> label ;
COM8 -> label ;
COM1 -> label ;


  0%|          | 0/8 [00:00<?, ?it/s]

((0.7733644859813084,
  0.7815126050420168,
  0.7758695711294226,
  0.6572438162544169),
 <pgmpy.base.DAG.DAG at 0x24bad7c8f40>)

In [21]:
caculate_label_ancsetor(dag_struct_init_list[c_id ],c_id )

COM7 -> COM8 ;
COM10 -> label ;
age -> COM1 ;
COM8 -> label ;
COM1 -> label ;


  0%|          | 0/57 [00:00<?, ?it/s]

((0.7733644859813084,
  0.7815126050420168,
  0.7758695711294226,
  0.6572438162544169),
 <pgmpy.base.DAG.DAG at 0x24bad7c8fa0>)

In [22]:
caculate_label_ancsetor(clients[c_id ].model,c_id )

COM7 -> label ;
COM7 -> COM8 ;
COM10 -> label ;
COM10 -> COM11 ;
age -> label ;
age -> LAB19 ;
age -> COM1 ;
COM1 -> label ;
COM4 -> label ;
COM8 -> label ;
COM11 -> label ;
LAB8 -> LAB19 ;
COM3 -> label ;
LAB19 -> label ;


  0%|          | 0/233 [00:00<?, ?it/s]

((0.8200934579439252,
  0.680672268907563,
  0.7772293383372767,
  0.6778242677824268),
 <pgmpy.base.DAG.DAG at 0x24bb07dad90>)

In [23]:
caculate_label_par(clients[c_id ].model,c_id )

age -> label ;
LAB19 -> label ;
COM1 -> label ;
COM3 -> label ;
COM4 -> label ;
COM7 -> label ;
COM8 -> label ;
COM10 -> label ;
COM11 -> label ;


  0%|          | 0/179 [00:00<?, ?it/s]

((0.8200934579439252,
  0.680672268907563,
  0.7772293383372767,
  0.6778242677824268),
 <pgmpy.base.DAG.DAG at 0x24bb44b4d00>)

In [24]:
hos_id = [420,142,122,435,390,227,144,140,396,141]

In [25]:
out_data =[]
for i in out_id:
    out_data.append(DB_1[DB_1["hospitalid"]==i].drop(columns="hospitalid",axis=1))
out_score = []
for i in out_data:
    out_score.append(net_score(fuseDAG,i))
pd.DataFrame(out_score,index = ["BN8","BN9","BN10"],columns=["acc","recall","auc","f1"]).T    

NameError: name 'out_id' is not defined

In [162]:
def caculate_mi(data):    
    mi_score_list = []
    h_id = 6
    for col in data.columns:
        for index in data.columns:
            if col==index:
                continue
            mi_score = mutual_info_score(data.loc[:,index],data.loc[:,col])
            mi_score_list.append([index,col,mi_score])
            
            ##互信息分数，如果两个特征的互信息分数小雨0.02的话就将它加入黑名单
   
    return pd.DataFrame(mi_score_list,columns=["form","to","mi"])

In [157]:
data_mi = caculate_mi(clients[2].data)
data_mi = data_mi.sort_values(by="mi",ascending=False)

In [222]:
data_mi

,form,sto,mi
2145,LAB44,LAB43,2.843245e-01
2209,LAB43,LAB44,2.843245e-01
2276,LAB48,LAB46,2.410375e-01
2403,LAB46,LAB48,2.410375e-01
2535,LAB54,LAB53,2.387234e-01
...,...,...,...
3255,PRO1,COM6,3.654938e-08
3102,LAB38,COM4,5.002825e-09
1967,COM4,LAB38,5.002825e-09
2734,COM3,LAB56,2.346444e-09


In [218]:
test_dag = DAG()
for i in range(len(data_mi)):
    if data_mi.iloc[i,1]=="label":
#         print(data_mi.iloc[i,0],data_mi.iloc[i,1],data_mi.iloc[i,2])
        if data_mi.iloc[i,2] > 0.015:
            test_dag.add_edge(data_mi.iloc[i,0],data_mi.iloc[i,1])

In [219]:
# test_dag.remove_edge("COM1","label")

In [220]:
print_dag(test_dag)

COM10 -> label ;
COM8 -> label ;
COM11 -> label ;
LAB19 -> label ;
COM3 -> label ;


In [221]:
net_score(test_dag,clients[6].data)

  0%|          | 0/27 [00:00<?, ?it/s]

(0.7359307359307359,
 0.38666666666666666,
 0.6452564102564103,
 0.48739495798319327)

In [ ]:
['fnml','bdla','mbde','bds','bic','loglik','bde', 'mbde', 'k2', 'qnml']

In [25]:
out_data =[]
for i in hos_id:
    out_data.append(DB_1[DB_1["hospitalid"]==i].drop(columns="hospitalid",axis=1))
label_dag = DAG(fuseDAG.get_ancestral_graph("label"))
label_dag.add_edge("sex","label")
label_dag.add_edge("race","label")
out_score = []
for i in out_data:
    out_score.append(net_score(label_dag,i))
pd.DataFrame(out_score,columns=["acc","recall","auc","f1"]).T

  0%|          | 0/552 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/414 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

  0%|          | 0/193 [00:00<?, ?it/s]

...

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/289 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9
acc,0.845794,0.797546,0.778226,0.782609,0.791304,0.811138,0.766234,0.815287,0.797546,0.806075
recall,0.691892,0.620915,0.531401,0.681416,0.721805,0.589286,0.506667,0.604167,0.577778,0.571429
auc,0.799994,0.749446,0.702386,0.752578,0.778355,0.741487,0.698846,0.756212,0.729567,0.733934
f1,0.721127,0.657439,0.571429,0.644351,0.727273,0.628571,0.584615,0.666667,0.611765,0.621005


In [26]:
all_score_list[0]

,BN1,BN2,BN3,BN4,BN5,BN6,BN7,BN8,BN9,BN10
acc,0.845794,0.809816,0.811828,0.774936,0.802899,0.801453,0.774892,0.834395,0.834356,0.820093
recall,0.705405,0.660131,0.599034,0.681416,0.796992,0.544643,0.613333,0.812500,0.844444,0.680672
auc,0.804016,0.769053,0.746444,0.747183,0.801798,0.720826,0.732949,0.828268,0.837476,0.777229
f1,0.725000,0.684746,0.639175,0.636364,0.757143,0.598039,0.638889,0.750000,0.737864,0.677824


In [ ]:
def get_label_str(cl,tag=1):
    arc_str=cl.arc_strength.sort_values(by="strength",ascending=False)
    label_str = []
    if tag:
        for ed in cl.model.get_ancestral_graph("label").edges():
            s=arc_str[(arc_str["from"]==ed[0])&(arc_str["to"]==ed[1])]["strength"]
            label_str.append([ed[0],ed[1],s[0]])
        label_str=pd.DataFrame(label_str,columns=["from","to","strength"])
        return label_str
    else:
        for ed in cl.model.edges():
            s=arc_str[(arc_str["from"]==ed[0])&(arc_str["to"]==ed[1])]["strength"]
            label_str.append([ed[0],ed[1],s[0]])
        label_str=pd.DataFrame(label_str,columns=["from","to","strength"])
    return label_str.sort_values(by="strength",ascending=False)

In [ ]:
ho_id=2
get_label_str(clients[ho_id]).sort_values(by="strength",ascending=False)

In [ ]:

mi_df = []
for ed in clients[ho_id].model.get_ancestral_graph("label").edges():
    mi_score = mutual_info_score(clients[ho_id].data.loc[:,ed[0]],clients[ho_id].data.loc[:,ed[1]])
    mi_df.append([ed[0],ed[1],mi_score])
mi_df=pd.DataFrame(mi_df,columns=["from","to","mi_score"])
mi_df.sort_values(by="mi_score",ascending=False)

In [ ]:
for i in range(10):
    print(set(clients[i].data["PRO5"]))

In [ ]:
auc_list = []
recall_list = []
acc_list = []
#测试更新后的性能
for i in range(10):
#     print("各个客户端的初始性能")
    acc,recall,auc_score = cross_validation(clients[i].model,client_data,clients[i].cid)
    auc_list.append(auc_score)
    recall_list.append(recall)
    acc_list.append(acc)
#     print(i)
#     print_dag(clients[i].model)
# print("更新auc",auc_list)
# print("更新recall",recall_list)
# print("更新acc",acc_list)

    
    